## groupBy()

- groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

- GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [ ]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()

- 지역별 교육타입별(cdf.loc, cdf.teaching_type)) 학생 숫자(class_std_cnt)를 구해보자.


In [ ]:
# cdf.groupby(cdf.loc, cdf.teaching_type).sum('class_std_cnt').show()
# AnalysisException: "class_std_cnt" is not a numeric column.
# class_std_cnt가 문자열이라 문제가 된다.
# 스키마를 적용해서 해결 해보자

In [1]:
# 스키마 설정 후 dataload
schema = StructType([
        StructField('class_cd',StringType()),
        StructField('school',StringType()),
        StructField('class_std_cnt',IntegerType()),    
        StructField('loc',StringType()),
        StructField('school_type',StringType()),
        StructField('teaching_type',StringType())    
])

cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True, schema=schema)
cdf.printSchema()


In [ ]:
# 지역에 따른 교육 타입별 학생수 추출
cdf.groupBy(cdf.loc,cdf.teaching_type).sum('class_std_cnt').show()

In [ ]:
print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')
cdf.where(cdf.loc.isNotNull()) # 지역정보가 있는 데이터만 추출
cdf.where(cdf.loc.isNotNull()).groupby(cdf.loc) # 지역별로 그룹핑(카테고리)
cdf.where(cdf.loc.isNotNull()).groupby(cdf.loc).count() # 카테고리별 원소수를 개수한 결과가 df로 반환
cdf.where(cdf.loc.isNotNull()).groupby(cdf.loc).count().show()

In [3]:
print('지역별 교육타입별 학생수의 합과 평균을 구해보자. 단 학생수의 합이 300미만인 데이터는 제외한다.')
# 학생수의 합과 평균 : 두개의 집계함수를 동시에 사용해야 함 => agg() 사용해서 해결
cdf.groupby(cdf.loc, cdf.teaching_type)\
    .agg(sum('class_std_cnt'),avg('class_std_cnt')).show(2) #지역별 교육타입별 학생수의 합과 평균을 구해보자

cdf.groupby(cdf.loc, cdf.teaching_type)\
    .agg(sum('class_std_cnt'),avg('class_std_cnt'))\
    .where(col('sum(class_std_cnt)')>=300).show() #단 학생수의 합이 300미만인 데이터는 제외한다, agg 함수에서 계산한 결과를 활용해서 300이상인지 체크

cdf.groupby(cdf.loc, cdf.teaching_type)\
    .agg(sum('class_std_cnt'),avg('class_std_cnt'))\
    .where(sum('class_std_cnt')>=300).show() # groupby된 객체를 이용해서 학생수의 합을 다시계산, agg에서 합을 계산, where에서 합을 계산

# cdf.groupby(cdf.loc, cdf.teaching_type)\
#     .agg(sum('class_std_cnt'),avg('class_std_cnt'))\
#     .where('sum(class_std_cnt)'>=300)  # 컴럼명을 문자열로만 전달하는건 where/filter에서는 사용할 수 없다

In [ ]:
print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

cdf.groupby(cdf.loc, cdf.teaching_type)\
    .agg(sum('class_std_cnt').alias('total'),avg('class_std_cnt').alias('avg'))\
    .where(col('total')>=300).show()

In [14]:
# 지역별로 구한 학생 수 총합에 대해 학생이 가장많은 지역과 가장 적은 지역의 차이를 구해보자

cdf.where(cdf.loc.isNotNull())\
   .groupby(cdf.loc)\
   .agg(sum('class_std_cnt').alias('student_tot')) # dataframe 반환, 연결 method max,min등의 집계함수는 연결 불가

cdf.where(cdf.loc.isNotNull())\
   .groupby(cdf.loc)\
   .agg(sum('class_std_cnt').alias('student_tot'))\
   .select(max(col('student_tot'))-min(col('student_tot')))\
   .show()



+---------------------+
|(max(tot) - min(tot))|
+---------------------+
|                  368|
+---------------------+



In [ ]:
# groupby  사용 가능 집계함수
cdf.where(cdf.loc.isNotNull())\
   .groupby(cdf.loc)\
   .agg(sum('class_std_cnt').alias('sum'),
        avg('class_std_cnt').alias('avg'),
        max('class_std_cnt').alias('max'),
        min('class_std_cnt').alias('min'),
        stddev('class_std_cnt').alias('stddev'),
        count('class_std_cnt').alias('count')).show()

#### sql

In [4]:
# dataFrame을 테이블로 등록
cdf.createOrReplaceTempView('class')


print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')
# sql query의 groupby 절은 select 기준열이나 집계합수를 제외한 나머지는 표기 불가 from view/table group by 기준열 having 조건절
# spark.sql("select loc, count(*) as tot, class_std_cnt  from class group by loc having loc is not null;").show() # groupby 기준이 아닌 열을 select에 표기
spark.sql("select loc, count(*) as tot  from class group by loc having loc is not null;").show()

In [ ]:
print('''지역내 교육타입별 학생 수의 합과 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')
spark.sql('''select loc, sum(class_std_cnt) as tot, avg(class_std_cnt) as avg
          from class 
          group by loc, teaching_type
          having tot >= 300
          ''').show()

In [ ]:
print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

spark.sql('''select loc, sum(class_std_cnt) as tot, avg(class_std_cnt) as avg
             from class 
             group by loc, teaching_type
             having tot >= 300
             ''').show()

## orderBy()

In [5]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.class_std_cnt.desc()).show(1)

print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.loc.asc(), cdf.school.desc()).show(1)


In [ ]:
# 연습
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
# 값이 null 인 행우선 표시, asc_nulls_first()
cdf.orderBy(cdf.school_type.asc()).show(1)
cdf.orderBy(cdf.school_type.asc_nulls_first()).show(1)
cdf.orderBy(cdf.school_type.desc_nulls_first()).show(1)

- orderBy(컬럼 객체, asc()/컬럼객체.desc())
- 컬럼객체의 sort exp 메서드
    - asc() : 오름차순
    - desc() : 내림차순
    - asc_nulls_first()/desc_null_first() : null값이 있으면 우선 위에 표시하고 null 이 아닌 data 는 오름/내림차순 정렬

In [ ]:
# 연습

print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
# 값이 null인 행우선 표시. asc_nulls_first()
cdf.orderBy(cdf.school_type.asc())
cdf.orderBy(cdf.school_type.asc_nulls_first())
cdf.orderBy(cdf.school_type.desc_nulls_first()).show()

#### sql

In [ ]:
print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql('select * from class order by loc asc, school desc').show(3)

In [ ]:
# 연습
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
    select * from class order by school_type asc nulls first    
''').show(3)

In [ ]:
# 연습
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
    select * from class order by school_type asc
''')

spark.sql('''
    select * from class order by school_type asc nulls first
''').show(3)

In [ ]:
# 학교 종류를 기준으로 내림차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라
spark.sql('''
    select * from class order by school_type desc nulls first
''').show(10)